In [1]:
import timeit

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def read_unstructured_field_from_file(filename, num_halo=None):
    (rank, nbits, nx, ny, nz) = np.fromfile(filename, dtype=np.int32, count=5)
    offset = (3 + rank) * 32 // nbits
    data = np.fromfile(
        filename,
        dtype=np.float32 if nbits == 32 else np.float64)
    if rank == 3:
        return np.reshape(data[5:], (nz, ny, nx))
    else:
        return np.reshape(data[5:], (ny, nx))


def validate_unstructured_results(in_filename, out_filename):
    fig, axs = plt.subplots(1, 2, figsize=(12, 4))

    in_field = read_unstructured_field_from_file(in_filename)
    print(in_field.shape[0])
    im1 = axs[0].imshow(
        in_field[in_field.shape[0] // 2, :, :], origin="lower", vmin=-0.1, vmax=1.1
    )
    fig.colorbar(im1, ax=axs[0])
    axs[0].set_title("Initial condition")

    out_field = read_unstructured_field_from_file(out_filename)
    im2 = axs[1].imshow(
        out_field[out_field.shape[0] // 2, :, :], origin="lower", vmin=-0.1, vmax=1.1
    )
    fig.colorbar(im2, ax=axs[1])
    axs[1].set_title("Final result")

    plt.show()

# Compile sequential

In [2]:
%%bash
module load daint-gpu
module switch PrgEnv-gnu PrgEnv-cray
module load perftools-lite

# make clean
make VERSION=sequential_nPoint_4

make: 'stencil2d-sequential_nPoint_4.x' is up to date.


In [3]:
%%bash
srun -n 1 ./stencil2d-sequential_nPoint_4.x+orig --nx 128 --ny 128 --nz 64 --num_iter 1024

# ranks nx ny nz num_iter time
data = np.array( [ \
[    1,  128,  128,   64,    1024,  0.4548142E+01], \
] )


# Compile random

In [4]:
%%bash
module load daint-gpu
module switch PrgEnv-gnu PrgEnv-cray
module load perftools-lite

# make clean
make VERSION=random_nPoint_4

make: 'stencil2d-random_nPoint_4.x' is up to date.


In [5]:
%%bash
srun -n 1 ./stencil2d-random.x+orig --nx 128 --ny 128 --nz 64 --num_iter 1024

# ranks nx ny nz num_iter time
data = np.array( [ \
[    1,  128,  128,   64,    1024,  0.5950273E+01], \
] )


# Compile filling curve

In [6]:
%%bash
module load daint-gpu
module switch PrgEnv-gnu PrgEnv-cray
module load perftools-lite

# make clean
make VERSION=hilbert_nPoint_4

make: 'stencil2d-hilbert_nPoint_4.x' is up to date.


In [ ]:
%%bash
srun -n 1 ./stencil2d-hilbert.x+orig --nx 128 --ny 128 --nz 64 --num_iter 1024

In [7]:
# Take a note of time, restart the Kernel, and repeat 10 times.

SyntaxError: invalid syntax (3080853175.py, line 1)